In [2]:
# GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
  
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

# TIME
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Seoul /etc/localtime
!date

# GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Fri Feb 11 07:20:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!git clone https://github.com/baek2sm/ml.git

Cloning into 'ml'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 252 (delta 118), reused 194 (delta 72), pack-reused 0
Receiving objects: 100% (252/252), 64.48 MiB | 28.13 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import joblib

In [6]:
# 서울 기상 데이터세트를 불러옵니다.
datasets = joblib.load('./ml/datasets/weather.pickle')

# 입력 데이터와 타깃을 준비합니다.
data, target = datasets['data'], datasets['target']

# 입력 데이터와 타깃의 형태를 출력합니다.
print(data.shape, target.shape)

(26181, 24, 3) (26181,)


In [7]:
# 학습 세트와 테스트 세트를 나눕니다.
train_length = 20000
X_train, X_test = data[:train_length], data[train_length:]
y_train, y_test = target[:train_length], target[train_length:]

# 넘파이 자료구조를 텐서 자료구조로 변환합니다.
X_train, X_test = torch.from_numpy(X_train), torch.from_numpy(X_test)
y_train, y_test = torch.from_numpy(y_train), torch.from_numpy(y_test)

In [8]:
# 데이터세트 자료구조를 준비합니다.
dset_train, dset_test = TensorDataset(X_train, y_train), TensorDataset(X_test, y_test)

# 데이터 로더를 준비합니다.
train_loader = DataLoader(dset_train, batch_size=256, shuffle=True)
test_loader = DataLoader(dset_test, batch_size=256, shuffle=False)

In [11]:
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cell = nn.RNN(3,3, batch_first=True)
        self.fc = nn.Linear(24*3, 1)

    def forward(self, X):
        out, hidden_state = self.cell(X)
        out = out.contiguous()
        out = self.fc(out.view(-1, 24*3))
        return out

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = RNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=2e-4)

In [14]:
def train(model, criterion, optimizer, loader):
    epoch_loss = 0
    model.train()

    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(loader)


In [15]:
def evaluate(model, cirterion, loader):
    epoch_loss = 0
    model.eval()
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
            hypothesis = model(X_batch)
            loss = criterion(hypothesis, y_batch)
            epoch_loss += loss.item()
        return epoch_loss /len(loader)

In [25]:
for epoch in range(1, 201):
    loss = train(model, criterion, optimizer, train_loader)
    test_loss = evaluate(model, criterion, test_loader)
    if epoch%20==0:
        print('epoch: {:3d}, loss: {:2.3f}, test_loss: {:.3f}'.format(epoch, loss, test_loss))

epoch:  20, loss: 9.448, test_loss: 11.290
epoch:  40, loss: 9.387, test_loss: 11.290
epoch:  60, loss: 9.326, test_loss: 11.297
epoch:  80, loss: 9.260, test_loss: 11.204
epoch: 100, loss: 9.327, test_loss: 11.188
epoch: 120, loss: 9.257, test_loss: 11.192
epoch: 140, loss: 9.240, test_loss: 11.170
epoch: 160, loss: 9.167, test_loss: 11.061
epoch: 180, loss: 9.098, test_loss: 11.106
epoch: 200, loss: 9.076, test_loss: 11.078


In [31]:
from math import hypot
model.eval()
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
        hypothesis = model(X_batch)
        print('predicted target: {:2.2f}, real target: {:3.2f} \t\t오차 {:2.2f}'.format(hypothesis[0].item(), y_batch[0].item(), abs(y_batch[0].item()-hypothesis[0].item())))

predicted target: 18.19, real target: 18.40 		오차 0.21
predicted target: 7.89, real target: 5.20 		오차 2.69
predicted target: 12.24, real target: 11.50 		오차 0.74
predicted target: 5.56, real target: 8.90 		오차 3.34
predicted target: 2.49, real target: -0.20 		오차 2.69
predicted target: -1.77, real target: 0.20 		오차 1.97
predicted target: 1.95, real target: 0.30 		오차 1.65
predicted target: 0.71, real target: 5.00 		오차 4.29
predicted target: 1.93, real target: -1.20 		오차 3.13
predicted target: 2.76, real target: 3.60 		오차 0.84
predicted target: 3.19, real target: 2.70 		오차 0.49
predicted target: 0.81, real target: 5.20 		오차 4.39
predicted target: 6.25, real target: 8.20 		오차 1.95
predicted target: 0.75, real target: 0.50 		오차 0.25
predicted target: 8.06, real target: 3.60 		오차 4.46
predicted target: 14.20, real target: 16.00 		오차 1.80
predicted target: 5.00, real target: 7.30 		오차 2.30
predicted target: 16.08, real target: 13.20 		오차 2.88
predicted target: 12.11, real target: 15.80 		오차 3.69